# 📊 PHÂN TÍCH BỔ SUNG - MDA2025

**3 phân tích còn thiếu:**
1. Data Cleaning
2. Factor Analysis
3. Regression Analysis

---

## 📌 HƯỚNG DẪN SỬ DỤNG

### Option A: Chạy notebook này riêng
1. Upload file `onlinebuy.csv`
2. Chạy Cell 0 để đọc dữ liệu
3. Chạy Cell 1, 2, 3 theo thứ tự

### Option B: Copy vào notebook chính
1. Mở `Phân_tích_marketing_UPDATED.ipynb`
2. Tìm vị trí sau cell đọc dữ liệu
3. Copy Cell 1, 2, 3 từ đây
4. Paste vào notebook chính

In [ ]:
# ============================================================
# CELL 0: ĐỌC DỮ LIỆU (chỉ cần nếu chạy notebook này riêng)
# ============================================================

import pandas as pd
import numpy as np

# Đọc dữ liệu
df = pd.read_csv('onlinebuy.csv')
print(f"Kích thước dữ liệu gốc: {df.shape}")
df.head()

In [ ]:
# ============================================================
# CELL 1: DATA CLEANING
# ============================================================

print("="*80)
print("🧹 BƯỚC 1: DATA CLEANING")
print("="*80)

# Danh sách các cột Likert cần kiểm tra
likert_cols = [
    'int1','int2','inf1','inf2','inf3','ve1','ve2','ve3','nvse1','nvse2',
    'trust1','trust2','trust3','conv1','conv2','conv3','conv4',
    'enj1','enj2','enj3','sc1','sc2','al1','al2','al3'
]

def check_invalid(row):
    """
    Kiểm tra dữ liệu không hợp lệ:
    1. Tất cả giá trị giống nhau
    2. ≥10 giá trị liên tiếp giống nhau
    """
    vals = row[likert_cols].values
    
    # Kiểm tra tất cả giống nhau
    if len(set(vals)) == 1:
        return True
    
    # Kiểm tra 10 liên tiếp giống nhau
    max_c = current = 1
    for i in range(1, len(vals)):
        if vals[i] == vals[i-1]:
            current += 1
            max_c = max(max_c, current)
        else:
            current = 1
    return max_c >= 10

# Tìm và loại bỏ dữ liệu không hợp lệ
print(f"\n🔍 Kiểm tra dữ liệu...")
invalid = [i for i, row in df.iterrows() if check_invalid(row)]

print(f"\n📊 KẾT QUẢ:")
print(f"   Tổng mẫu ban đầu: {len(df)}")
print(f"   Mẫu không hợp lệ: {len(invalid)} ({len(invalid)/len(df)*100:.1f}%)")

if len(invalid) > 0:
    print(f"\n   Ví dụ 5 mẫu đầu tiên bị loại: {invalid[:5]}")

# Loại bỏ
df = df.drop(invalid).reset_index(drop=True)

print(f"\n✅ SAU KHI LÀM SẠCH:")
print(f"   Còn lại: {len(df)} mẫu ({len(df)/(len(df)+len(invalid))*100:.1f}%)")
print(f"   Đã loại: {len(invalid)} mẫu")

# Lưu dữ liệu sạch
df.to_csv('onlinebuy_cleaned.csv', index=False)
print(f"\n💾 Đã lưu: onlinebuy_cleaned.csv")

In [ ]:
# ============================================================
# CELL 2: FACTOR ANALYSIS
# ============================================================

print("\n" + "="*80)
print("📊 BƯỚC 2: FACTOR ANALYSIS")
print("="*80)

# Cài đặt thư viện
!pip install factor-analyzer -q

from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_kmo, calculate_bartlett_sphericity

# KMO Test
print("\n🔍 KMO & Bartlett's Test:")
kmo_all, kmo = calculate_kmo(df[likert_cols])
print(f"\n   KMO Score: {kmo:.3f}")

if kmo >= 0.9:
    print(f"   → ✅ Tuyệt vời (≥0.9)")
elif kmo >= 0.8:
    print(f"   → ✅ Rất tốt (≥0.8)")
elif kmo >= 0.7:
    print(f"   → ✅ Trung bình (≥0.7)")
elif kmo >= 0.6:
    print(f"   → ⚠️  Vừa phải (≥0.6)")
else:
    print(f"   → ❌ Không phù hợp (<0.6)")

# Bartlett's Test
chi2, pval = calculate_bartlett_sphericity(df[likert_cols])
print(f"\n   Bartlett's Test:")
print(f"     Chi-square: {chi2:.2f}")
print(f"     p-value: {pval:.4f}")
if pval < 0.05:
    print(f"   → ✅ Có ý nghĩa (p < 0.05)")
else:
    print(f"   → ❌ Không có ý nghĩa (p ≥ 0.05)")

# Xác định số nhân tố tối ưu
print("\n🔍 Xác định số nhân tố:")
fa_test = FactorAnalyzer(n_factors=25, rotation=None)
fa_test.fit(df[likert_cols])
ev, v = fa_test.get_eigenvalues()
n_factors = sum(ev > 1.0)
print(f"\n   Số nhân tố có eigenvalue > 1.0: {n_factors}")
print(f"\n   Top 10 Eigenvalues:")
for i, eigenvalue in enumerate(ev[:10], 1):
    marker = "✅" if eigenvalue > 1.0 else "  "
    print(f"   {marker} Factor {i}: {eigenvalue:.3f}")

# Chạy Factor Analysis với Varimax
print(f"\n🔍 Factor Analysis với {n_factors} factors (Varimax Rotation):")
fa = FactorAnalyzer(n_factors=n_factors, rotation='varimax')
fa.fit(df[likert_cols])

# Factor Loadings
loadings_df = pd.DataFrame(
    fa.loadings_,
    index=likert_cols,
    columns=[f'Factor{i+1}' for i in range(n_factors)]
)

print("\n📋 Factor Loadings (chỉ hiển thị |loading| ≥ 0.4):")
for col in loadings_df.columns:
    high = loadings_df[col][abs(loadings_df[col]) >= 0.4].sort_values(ascending=False)
    if len(high) > 0:
        print(f"\n   {col}:")
        for var, val in high.items():
            print(f"     {var:8s}: {val:6.3f}")

# Variance Explained
variance = fa.get_factor_variance()
print(f"\n📊 Variance Explained:")
print(f"   Tổng phương sai giải thích: {variance[2][-1]*100:.1f}%")

# Lưu kết quả
loadings_df.to_csv('factor_loadings.csv')
print(f"\n💾 Đã lưu: factor_loadings.csv")

# Hiển thị toàn bộ loadings matrix
print("\n📋 FULL FACTOR LOADINGS MATRIX:")
display(loadings_df.round(3))

In [ ]:
# ============================================================
# CELL 3: REGRESSION ANALYSIS
# ============================================================

print("\n" + "="*80)
print("📈 BƯỚC 3: REGRESSION ANALYSIS")
print("="*80)

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Tạo biến AL (Attitudinal Loyalty)
print("\n🔧 Tạo biến AL (Attitudinal Loyalty)...")
df['AL'] = df[['al1','al2','al3']].mean(axis=1)
print(f"   ✅ AL = mean(al1, al2, al3)")
print(f"   Mean AL: {df['AL'].mean():.3f}")
print(f"   Std AL: {df['AL'].std():.3f}")

# ============================================================
# REGRESSION 1: Platform Features → Attitudinal Loyalty
# ============================================================

print("\n" + "-"*80)
print("📊 REGRESSION 1: Platform Features → Attitudinal Loyalty")
print("-"*80)

# Biến độc lập: Platform characteristics
X1_vars = ['INT', 'INF', 'VE', 'NVSE']
X1 = df[X1_vars]
y = df['AL']

# Thêm constant
X1_const = sm.add_constant(X1)

# Chạy OLS
m1 = sm.OLS(y, X1_const).fit()

print(f"\n📈 KẾT QUẢ MODEL 1:")
print(f"   R² = {m1.rsquared:.4f}")
print(f"   Adjusted R² = {m1.rsquared_adj:.4f}")
print(f"   F-statistic = {m1.fvalue:.4f} (p = {m1.f_pvalue:.4f})")

print(f"\n   Coefficients:")
for var in ['const'] + X1_vars:
    coef = m1.params[var]
    pval = m1.pvalues[var]
    sig = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else ""
    print(f"     {var:8s}: {coef:7.4f}  (p = {pval:.4f}) {sig}")

print(f"\n   Ghi chú: *** p<0.001, ** p<0.01, * p<0.05")

# VIF Check
print(f"\n   🔍 VIF (Multicollinearity Check):")
vif_data1 = pd.DataFrame()
vif_data1["Variable"] = X1_vars
vif_data1["VIF"] = [variance_inflation_factor(X1.values, i) for i in range(len(X1_vars))]
print(vif_data1.to_string(index=False))
print(f"\n   VIF < 5: ✅ Không có vấn đề")
print(f"   VIF 5-10: ⚠️ Cần xem xét")
print(f"   VIF > 10: ❌ Đa cộng tuyến nghiêm trọng")

# Full summary
print(f"\n📋 FULL REGRESSION SUMMARY:")
print(m1.summary())

# ============================================================
# REGRESSION 2: Psychological Factors → Attitudinal Loyalty
# ============================================================

print("\n" + "-"*80)
print("📊 REGRESSION 2: Psychological Factors → Attitudinal Loyalty")
print("-"*80)

# Biến độc lập: Psychological responses
X2_vars = ['TRUST', 'CONV', 'ENJ', 'SC']
X2 = df[X2_vars]

# Thêm constant
X2_const = sm.add_constant(X2)

# Chạy OLS
m2 = sm.OLS(y, X2_const).fit()

print(f"\n📈 KẾT QUẢ MODEL 2:")
print(f"   R² = {m2.rsquared:.4f}")
print(f"   Adjusted R² = {m2.rsquared_adj:.4f}")
print(f"   F-statistic = {m2.fvalue:.4f} (p = {m2.f_pvalue:.4f})")

print(f"\n   Coefficients:")
for var in ['const'] + X2_vars:
    coef = m2.params[var]
    pval = m2.pvalues[var]
    sig = "***" if pval < 0.001 else "**" if pval < 0.01 else "*" if pval < 0.05 else ""
    print(f"     {var:8s}: {coef:7.4f}  (p = {pval:.4f}) {sig}")

print(f"\n   Ghi chú: *** p<0.001, ** p<0.01, * p<0.05")

# VIF Check
print(f"\n   🔍 VIF (Multicollinearity Check):")
vif_data2 = pd.DataFrame()
vif_data2["Variable"] = X2_vars
vif_data2["VIF"] = [variance_inflation_factor(X2.values, i) for i in range(len(X2_vars))]
print(vif_data2.to_string(index=False))

# Full summary
print(f"\n📋 FULL REGRESSION SUMMARY:")
print(m2.summary())

# ============================================================
# SO SÁNH 2 MODELS
# ============================================================

print("\n" + "="*80)
print("📊 SO SÁNH 2 MODELS")
print("="*80)

comparison = pd.DataFrame({
    'Model': ['Model 1: Platform', 'Model 2: Psychology'],
    'R²': [m1.rsquared, m2.rsquared],
    'Adj. R²': [m1.rsquared_adj, m2.rsquared_adj],
    'F-stat': [m1.fvalue, m2.fvalue],
    'Prob(F)': [m1.f_pvalue, m2.f_pvalue]
})

print("\n" + comparison.to_string(index=False))

# Lưu kết quả
results1 = pd.DataFrame({
    'Variable': m1.params.index,
    'Coefficient': m1.params.values,
    'Std Error': m1.bse.values,
    'p-value': m1.pvalues.values
})
results1.to_csv('regression1_results.csv', index=False)

results2 = pd.DataFrame({
    'Variable': m2.params.index,
    'Coefficient': m2.params.values,
    'Std Error': m2.bse.values,
    'p-value': m2.pvalues.values
})
results2.to_csv('regression2_results.csv', index=False)

print(f"\n💾 Đã lưu:")
print(f"   - regression1_results.csv")
print(f"   - regression2_results.csv")

print("\n" + "="*80)
print("✅ HOÀN THÀNH TẤT CẢ PHÂN TÍCH!")
print("="*80)

---

## 🎉 KẾT QUẢ

Sau khi chạy 3 cells trên, bạn đã có:

### ✅ Files được tạo:
1. `onlinebuy_cleaned.csv` - Dữ liệu đã làm sạch
2. `factor_loadings.csv` - Kết quả Factor Analysis
3. `regression1_results.csv` - Kết quả Regression 1
4. `regression2_results.csv` - Kết quả Regression 2

### ✅ Kết quả in ra:
- Số mẫu bị loại bỏ
- KMO Score
- Số factors tối ưu
- Factor loadings matrix
- R² cho 2 models
- Coefficients với p-values
- VIF scores

### 📝 Bước tiếp theo:
1. Copy kết quả vào báo cáo Word
2. Tìm 15+ references
3. Viết báo cáo 4,000 từ
4. Submit!

---

**💪 GOOD LUCK!**